<a href="https://colab.research.google.com/github/mrdbourke/pytorch-deep-learning/blob/main/extras/exercises/01_pytorch_workflow_exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 01. PyTorch Workflow Exercise Template

The following is a template for the PyTorch workflow exercises.

It's only starter code and it's your job to fill in the blanks.

Because of the flexibility of PyTorch, there may be more than one way to answer the question.

Don't worry about trying to be *right* just try writing code that suffices the question.

You can see one form of [solutions on GitHub](https://github.com/mrdbourke/pytorch-deep-learning/tree/main/extras/solutions) (but try the exercises below yourself first!).

In [12]:
# Import necessary libraries
import torch 
import torch.nn as nn 
from sklearn.model_selection import train_test_split

In [13]:
# Setup device-agnostic code
torch.manual_seed(42)

## 1. Create a straight line dataset using the linear regression formula (`weight * X + bias`).
  * Set `weight=0.3` and `bias=0.9` there should be at least 100 datapoints total. 
  * Split the data into 80% training, 20% testing.
  * Plot the training and testing data so it becomes visual.

Your output of the below cell should look something like:
```
Number of X samples: 100
Number of y samples: 100
First 10 X & y samples:
X: tensor([0.0000, 0.0100, 0.0200, 0.0300, 0.0400, 0.0500, 0.0600, 0.0700, 0.0800,
        0.0900])
y: tensor([0.9000, 0.9030, 0.9060, 0.9090, 0.9120, 0.9150, 0.9180, 0.9210, 0.9240,
        0.9270])
```

Of course the numbers in `X` and `y` may be different but ideally they're created using the linear regression formula.

In [14]:
# Create the data parameters
w=0.3
b=0.9
X=torch.arange(0,1,0.02).unsqueeze(dim=1)
y=w*X+b
# Make X and y using linear regression feature


print(f"Number of X samples: {len(X)}")
print(f"Number of y samples: {len(y)}")
print(f"First 10 X & y samples:\nX: {X[:10]}\ny: {y[:10]}")

Number of X samples: 50
Number of y samples: 50
First 10 X & y samples:
X: tensor([[0.0000],
        [0.0200],
        [0.0400],
        [0.0600],
        [0.0800],
        [0.1000],
        [0.1200],
        [0.1400],
        [0.1600],
        [0.1800]])
y: tensor([[0.9000],
        [0.9060],
        [0.9120],
        [0.9180],
        [0.9240],
        [0.9300],
        [0.9360],
        [0.9420],
        [0.9480],
        [0.9540]])


In [15]:
# Split the data into training and testing
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=42,test_size=0.2)

In [16]:
# Plot the training and testing data 


## 2. Build a PyTorch model by subclassing `nn.Module`. 
  * Inside should be a randomly initialized `nn.Parameter()` with `requires_grad=True`, one for `weights` and one for `bias`. 
  * Implement the `forward()` method to compute the linear regression function you used to create the dataset in 1. 
  * Once you've constructed the model, make an instance of it and check its `state_dict()`.
  * **Note:** If you'd like to use `nn.Linear()` instead of `nn.Parameter()` you can.

In [17]:
device="cuda" if torch.cuda.is_available() else "gpu"
device

'cuda'

In [8]:
# Create PyTorch linear regression model by subclassing nn.Module
class LineerRegressionModel(nn.Module):
    
    def __init__(self):
        super().__init__()
        input_features=100
        output_features=1
        hidden_layer_1=5
        hidden_layer_2=3
        
        self.linear_layer=nn.Sequential(
            nn.Linear(in_features=input_features,out_features=hidden_layer_1),
            nn.ReLU(),
            nn.Linear(in_features=hidden_layer_1,out_features=hidden_layer_2),
            nn.ReLU(),
            nn.Linear(in_features=hidden_layer_2,out_features=output_features),
            nn.Sigmoid().to(device=device)
        )
    def forward(self,x:nn.Module):
        return self.linear_layer(x)

In [9]:

# Instantiate the model and put it to the target device
model=LineerRegressionModel()
# model.to(device)
# next(model.parameters()).device

## 3. Create a loss function and optimizer using `nn.L1Loss()` and `torch.optim.SGD(params, lr)` respectively. 
  * Set the learning rate of the optimizer to be 0.01 and the parameters to optimize should be the model parameters from the model you created in 2.
  * Write a training loop to perform the appropriate training steps for 300 epochs.
  * The training loop should test the model on the test dataset every 20 epochs.

In [10]:
# Create the loss function and optimizer
optim=torch.optim.SGD(lr=0.01,params=model.parameters())
loss_func=nn.L1Loss()

In [11]:
X_train=X_train.to(device)

In [11]:
# Training loop

# Train model for 300 epochs
epochs=300

# Send data to target device
X_train=X_train.to(device)
X_test=X_test.to(device)
y_train=y_train.to(device)
y_test=y_test.to(device)

for epoch in range(epochs):
  ### Training

  # Put model in train mode
  model.train()

  # 1. Forward pass
  y_pred=model(X_train)

  # 2. Calculate loss
  loss=loss_func(y_pred,y_train)

  # 3. Zero gradients
  model.zero_grad()

  # 4. Backpropagation
  loss.backward()

  # 5. Step the optimizer
  optim.step()

  ### Perform testing every 20 epochs
  if epoch % 20 == 0:

    # Put model in evaluation mode and setup inference context
      model.eval()
      # 1. Forward pass
      with torch.inference_mode():
         test_pred=model(X_test)
      # 2. Calculate test loss
      test_loss=loss_func(test_pred,y_test)
      # Print out what's happening
      print(f"Epoch: {epoch} | Train loss: {loss:.3f} | Test loss: {test_loss:.3f}")

Epoch: 0 | Train loss: 0.161 | Test loss: 0.195
Epoch: 20 | Train loss: 0.105 | Test loss: 0.095
Epoch: 40 | Train loss: 0.086 | Test loss: 0.067
Epoch: 60 | Train loss: 0.075 | Test loss: 0.056
Epoch: 80 | Train loss: 0.064 | Test loss: 0.047
Epoch: 100 | Train loss: 0.052 | Test loss: 0.039
Epoch: 120 | Train loss: 0.041 | Test loss: 0.030
Epoch: 140 | Train loss: 0.030 | Test loss: 0.021
Epoch: 160 | Train loss: 0.019 | Test loss: 0.013
Epoch: 180 | Train loss: 0.008 | Test loss: 0.005
Epoch: 200 | Train loss: 0.002 | Test loss: 0.011
Epoch: 220 | Train loss: 0.002 | Test loss: 0.011
Epoch: 240 | Train loss: 0.002 | Test loss: 0.011
Epoch: 260 | Train loss: 0.002 | Test loss: 0.011
Epoch: 280 | Train loss: 0.002 | Test loss: 0.011


## 4. Make predictions with the trained model on the test data.
  * Visualize these predictions against the original training and testing data (**note:** you may need to make sure the predictions are *not* on the GPU if you want to use non-CUDA-enabled libraries such as matplotlib to plot).

In [12]:
# Make predictions with the model


In [13]:
# Plot the predictions (these may need to be on a specific device)


## 5. Save your trained model's `state_dict()` to file.
  * Create a new instance of your model class you made in 2. and load in the `state_dict()` you just saved to it.
  * Perform predictions on your test data with the loaded model and confirm they match the original model predictions from 4.

In [14]:
from pathlib import Path

# 1. Create models directory 


# 2. Create model save path 

# 3. Save the model state dict


In [15]:
# Create new instance of model and load saved state dict (make sure to put it on the target device)


In [16]:
# Make predictions with loaded model and compare them to the previous
